In [1]:
from datetime import datetime
import os
import numpy as np
import pandas as pd

# import functions
from pysrc.load_data import read_data, sort_data, transform_data, drop_data
from pysrc.load_spec import load_spec
from pysrc.summarize import summarize
from pysrc.remNaNs_spline import remNaNs_spline

In [2]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as psub

In [3]:
# # Config
# config = []
# for v in [
#     "PAYEMS", "JTSJOL", "CPIAUCSL", "DGORDER", "HSN1F", "RSAFS", "UNRATE", "HOUST", "INDPRO", "PPIFIS", "DSPIC96", "BOPTEXP", 'BOPTIMP', "WHLSLRIMSA", "TTLCONS", "IR", "CPILFESL", "PCEPILFE", "PCEPI", "PERMIT", "TCU", "BUSINV", "IQ", "GACDISA066MSFRBNY", "PCEC96", "GACDFSA066MSFRBPHI", "GDPC1", "ULCNFB", "A261RX1Q020SBEA"
# ]:
#     config.append({
#         "source_dataset_code": v,
#         "skip_country_codes": [],
#         "include_country_codes": []
#     })

# SQL
# select VariableCode, ReferenceDate, VariableValue 
#     from NewestData nd
#     left join Variable v on v.VariableId=nd.VariableId
#     where VariableCode in (
#         'PAYEMS', 'JTSJOL', 'CPIAUCSL', 'DGORDER', 'HSN1F', 'RSAFS', 'UNRATE', 'HOUST', 'INDPRO', 'PPIFIS', 'DSPIC96', 'BOPTEXP', 'BOPTIMP', 'WHLSLRIMSA', 'TTLCONS', 'IR', 'CPILFESL', 'PCEPILFE', 'PCEPI', 'PERMIT', 'TCU', 'BUSINV', 'IQ', 'GACDISA066MSFRBNY', 'PCEC96', 'GACDFSA066MSFRBPHI', 'GDPC1', 'ULCNFB', 'A261RX1Q020SBEA'
#     )

In [4]:
def source_data_prep(src_fpath, country="US"):
    df_long = pd.read_csv(src_fpath, decimal=",")
    df_long["ReferenceDate"] = pd.to_datetime(df_long["ReferenceDate"])
    df_pivot = df_long.pivot(index="ReferenceDate", columns="VariableCode", values="VariableValue")
    vintage = os.path.basename(src_fpath).split(".")[0]
    df_pivot.to_excel(os.path.join('data', country, f'{vintage}.xlsx'), sheet_name="data")
    return vintage

In [5]:
def conv_matrix_to_df(matrix, date_col):
    df = pd.DataFrame(matrix)

    headers = df.iloc[0].values
    df.columns = headers
    df = df.drop(index=0, axis=0)
    df["ReferenceDate"] = date_col
    return df.set_index("ReferenceDate").sort_index()

In [6]:
def load_data(datafile, Spec, sample=None, load_excel=False):
    """
    Load vintage of data from file and format as structure

    Parameters:
        datafile (str): Filename of Microsoft Excel workbook file
        Spec (dict): Model specification containing SeriesID and other info
        sample (float, optional): Sample period start date in numeric form
        load_excel (bool, optional): Flag to force loading from Excel

    Returns:
        X (np.ndarray): T x N numeric array, transformed dataset
        Time (np.ndarray): T x 1 numeric array, date number with observation dates
        Z (np.ndarray): T x N numeric array, raw (untransformed) dataset
    """
    print('Loading data...')

    ext = os.path.splitext(datafile)[1]  # file extension
    idx = datafile.rfind(os.path.sep)
    datafile_mat = os.path.join(datafile[:idx], 'mat', os.path.splitext(datafile[idx + 1:])[0] + '.npz')

    if os.path.exists(datafile_mat) and not load_excel:
        # Load raw data from a NumPy formatted binary (.npz) file
        with np.load(datafile_mat, allow_pickle=True) as data:
            Z = data['Z']
            Time = data['Time']
            Mnem = data['Mnem']
    elif ext in ['.xlsx', '.xls']:
        # Read raw data from Excel file
        Z, Time, Mnem = read_data(datafile)
        # np.savez(datafile_mat, Z=Z, Time=Time, Mnem=Mnem)
    else:
        raise ValueError('Only Microsoft Excel workbook files supported.')

    # Sort data based on model specification
    Z = sort_data(Z, Mnem, Spec)
    
    # Transform data based on model specification
    X, Time, Z, header = transform_data(Z, Time, Spec)

    # Drop data not in estimation sample
    if sample is not None:
        X, Time, Z = drop_data(X, Time, Z, sample)

    # Z = np.vstack([header, Z])
    # X = np.vstack([header, X])

    return X, Time, Z, header

In [7]:
from scipy.signal import lfilter
from scipy.interpolate import splrep, splev
import numpy as np

def filter(x, k):
    """Apply a moving average filter with a window size of 2*k+1."""
    numerator = np.ones(2*k+1) / (2*k+1)
    return lfilter(numerator, [1], x)

def remNaNs_spline(X,options):
    """
    Treats NaNs in the dataset for use in Dynamic Factor Models (DFM).

    This function processes NaNs in a data matrix `X` according to five cases, 
    which are useful for running functions in the `DFM.m` file that do not 
    accept missing value inputs.

    Replication files for: 
    "Nowcasting", 2010, by Marta Banbura, Domenico Giannone, and Lucrezia Reichlin, 
    in Michael P. Clements and David F. Hendry, editors, Oxford Handbook on Economic Forecasting.

    The software can be freely used in applications. Users are kindly requested to 
    add acknowledgments to published work and cite the above reference in any resulting publications.

    Args:
        X (ndarray): Input data matrix of shape (T, n) where `T` is time and `n` is the number of series.
        options (dict): A dictionary with the following keys:
            - method (int): Determines the method for handling NaNs.
                - 1: Replaces all missing values using a filter.
                - 2: Replaces missing values after removing trailing and leading zeros 
                     (a row is 'missing' if more than 80% is NaN).
                - 3: Only removes rows with leading and closing zeros.
                - 4: Replaces missing values after removing trailing and leading zeros 
                     (a row is 'missing' if all are NaN).
                - 5: Replaces missing values using a spline and then applies a filter.
            - k (int): Used in MATLAB's filter function for the 1-D filter. 
              Controls the rational transfer function's numerator, where the 
              denominator is set to 1. The numerator takes the form 
              `ones(2*k+1, 1) / (2*k+1)`. See MATLAB's documentation for `filter()` for details.

    Returns:
        tuple:
            - X (ndarray): The processed data matrix.
            - indNaN (ndarray): A matrix indicating the location of missing values (1 for NaN).
    """
    T, N = X.shape  # Gives dimensions for data input
    k = options["k"]  # Inputted options
    method = options["method"]  # Inputted options
    indNaN = np.isnan(X)  # Returns location of NaNs
    nanLE = None
    if method == 1:   # replace all the missing values
        for i in range(N):  # loop through columns
            x = X[:, i]
            isnanx = indNaN[:, i]
            x[isnanx]  = np.nanmedian(x)  # Replace missing values series median
            x_MA = filter(np.concatenate(([x[0]] * k, x, [x[-1]] * k)), k)  # Apply filter
            x_MA = x_MA[2*k:]  # Match dimensions
            x[isnanx] = x_MA[isnanx]  # Replace missing observations with filtered values
            X[:, i] = x  # Replace vector
    elif method == 2:   # replace missing values after removing leading and closing zeros
        rem1 = np.sum(indNaN, axis=1) > N * 0.8  # Returns row sum for NaN values. Marks true for rows with more than 80% NaN
        nanLead = np.cumsum(rem1) == np.arange(1, T+1)
        nanEnd = np.cumsum(rem1[::-1]) == np.arange(1, T+1)
        nanEnd = nanEnd[::-1]  # Reverses nanEnd
        nanLE = nanLead | nanEnd

        print(X.shape, X[~nanLE, :].shape)
        X = X[~nanLE, :]  # Remove leading and trailing NaN rows
        indNaN = np.isnan(X)  # Index for missing values

        # Loop for each series
        for i in range(N):
            x = X[:, i]
            isnanx = np.isnan(x)
            t1 = np.where(~isnanx)[0][0]  # First non-NaN entry
            t2 = np.where(~isnanx)[0][-1]  # Last non-NaN entry

            # Interpolates without NaN entries in beginning and end
            tck = splrep(np.where(~isnanx)[0], x[~isnanx], s=0)
            x[t1:t2+1] = splev(np.arange(t1, t2+1), tck)
            isnanx = np.isnan(x)

            x[isnanx] = np.nanmedian(x)  # Replace NaNs with the median

            # Apply filter
            x_MA = filter(np.concatenate(([x[0]] * k, x, [x[-1]] * k)), k)
            x_MA = x_MA[2*k:]
            x[isnanx] = x_MA[isnanx]
            X[:, i] = x

    elif method == 3:  # Only remove rows with leading and closing zeros
        rem1 = np.sum(indNaN, axis=1) == N
        nanLead = np.cumsum(rem1) == np.arange(1, T+1)
        nanEnd = np.cumsum(rem1[::-1]) == np.arange(1, T+1)
        nanEnd = nanEnd[::-1]
        nanLE = nanLead | nanEnd

        # Remove leading and trailing NaN rows
        X = X[~nanLE, :]
        indNaN = np.isnan(X)

    elif method == 4:  # Remove rows with leading and closing zeros & replace missing values
        rem1 = np.sum(indNaN, axis=1) == N
        nanLead = np.cumsum(rem1) == np.arange(1, T+1)
        nanEnd = np.cumsum(rem1[::-1]) == np.arange(1, T+1)
        nanEnd = nanEnd[::-1]
        nanLE = nanLead | nanEnd

        # Remove leading and trailing NaN rows
        X = X[~nanLE, :]
        indNaN = np.isnan(X)

        for i in range(N):
            x = X[:, i]
            isnanx = np.isnan(x)
            t1 = np.where(~isnanx)[0][0]
            t2 = np.where(~isnanx)[0][-1]

            # Interpolation
            tck = splrep(np.where(~isnanx)[0], x[~isnanx], s=0)
            x[t1:t2+1] = splev(np.arange(t1, t2+1), tck)
            isnanx = np.isnan(x)

            x[isnanx] = np.nanmedian(x)  # Replace NaNs with the median
            
            # Apply filter
            x_MA = filter(np.concatenate(([x[0]] * k, x, [x[-1]] * k)), k)
            x_MA = x_MA[2*k:]
            x[isnanx] = x_MA[isnanx]
            X[:, i] = x

    elif method == 5:  # Replace missing values
        indNaN = np.isnan(X)
        for i in range(N):
            x = X[:, i]
            isnanx = np.isnan(x)
            t1 = np.where(~isnanx)[0][0]
            t2 = np.where(~isnanx)[0][-1]

            # Interpolation
            tck = splrep(np.where(~isnanx)[0], x[~isnanx], s=0)
            x[t1:t2+1] = splev(np.arange(t1, t2+1), tck)
            isnanx = np.isnan(x)

            x[isnanx] = np.nanmedian(x)  # Replace NaNs with the median

            # Apply filter
            x_MA = filter(np.concatenate(([x[0]] * k, x, [x[-1]] * k)), k)
            x_MA = x_MA[2*k:]
            x[isnanx] = x_MA[isnanx]
            X[:, i] = x

    return X, indNaN, nanLE


In [8]:
## User inputs.
src_fpath = "./data/2024-08-26.csv"
vintage = source_data_prep(src_fpath=src_fpath)
# vintage = '2016-06-29'; # vintage dataset to use for estimation

country = 'US';         # United States macroeconomic data
sample_start = datetime.strptime('2000-01-01', '%Y-%m-%d'); # estimation sample


## Load model specification and dataset.
# Load model specification structure `Spec`
Spec = load_spec('Spec_US_example.xls');
# Parse `Spec`
SeriesID, SeriesName, Units, UnitsTransformed = Spec['seriesid'], Spec['seriesname'], Spec['units'], Spec['unitstransformed']

# Load data
datafile = os.path.join('data', country, f'{vintage}.xls') if os.path.exists(os.path.join('data', country, f'{vintage}.xls')) else os.path.join('data', country, f'{vintage}.xlsx');
X, Time, Z, header = load_data(datafile, Spec, sample_start);
summarize(X.astype(float),Time,Spec,vintage); # summarize data

Table 1: Model specification
              SeriesID                   SeriesName                 Units  \
0               PAYEMS           Payroll Employment  Thousands of Persons   
1               JTSJOL                 Job Openings             Thousands   
2             CPIAUCSL         Consumer Price Index                 Index   
3              DGORDER         Durable Goods Orders           $, Millions   
4                RSAFS                 Retail Sales           $, Millions   
5               UNRATE            Unemployment Rate                     %   
6                HOUST               Housing Starts    Thousands of Units   
7               INDPRO        Industrial Production                 Index   
8              DSPIC96              Personal Income   Chained $, Billions   
9              BOPTEXP                      Exports           $, Millions   
10             BOPTIMP                      Imports           $, Millions   
11             TTLCONS        Construction Spen

/Users/ejowik001/Desktop/Github/Nowcasting/pysrc/load_spec.py:41: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [9]:
# Prepare data -----------------------------------------------------------
Mx = np.nanmean(X, axis=0)
Wx = np.nanstd(X, axis=0)
xNaN = (X - Mx) / Wx  # Standardize series

optNaN = {"method": 2, "k": 3}
y_est, indNaN, nanLE = remNaNs_spline(xNaN, optNaN)

X_df = pd.DataFrame(data=y_est, columns=header, index=Time[~nanLE])
Z_df = pd.DataFrame(data=Z, columns=header, index=Time)

(296, 25) (295, 25)


In [36]:
fig = psub.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                            subplot_titles=("Raw Observed Data", "Transformed & Imputed Data"))

## Plot raw and transformed data.
# Industrial Production (INDPRO) <fred.stlouisfed.org/series/INDPRO>
series_name = "GDPC1"
idxSeries = SeriesID.index(series_name)
# Plot raw observed data
trace1 = go.Scatter(
    x=Z_df.index,
    y=Z_df[series_name],
    mode="markers" if Z_df[~nanLE][series_name].isna().sum() else "lines",
    name='Raw Observed Data',
    line=dict(color="#000000", width=1),  # #7BCC62 / #68b562 / #7BB562
    marker={"size": 4, "symbol": "diamond"},
)
fig.add_trace(trace1, row=1, col=1)

# Plot transformed data
trace2 = go.Scatter(
    x=X_df.index,
    y=X_df[series_name],
    mode='lines',
    name='Transformed Data',
    line=dict(color="#BDC1D6", width=1),  # #7BCC62 / #68b562 / #7BB562
)
fig.add_trace(trace2, row=2, col=1)
fig.update_layout(
    height=600,
    width=800,
    showlegend=False,
    title_text=series_name,
    plot_bgcolor="white",
)
fig.update_xaxes(range=[Time[0], Time[-1]], row=1, col=1, gridcolor="lightgrey")
fig.update_yaxes(title_text=Units[idxSeries], row=1, col=1, gridcolor="lightgrey")

fig.update_xaxes(range=[Time[0], Time[-1]], title_text='Time', row=2, col=1, gridcolor="lightgrey")
fig.update_yaxes(title_text=UnitsTransformed[idxSeries], row=2, col=1, gridcolor="lightgrey")
fig.show()